# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [59]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [60]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [61]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [62]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [63]:
# display csv data as dataframe
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


# Part II. Apache Cassandra coding portion of the project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [65]:
# Make a connection to a Cassandra instance
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [66]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_db
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [67]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify_db')
except Exception as e:
    print(e)

### Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [68]:
# show df 
df.head(1)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88


### QUERY 1

Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

The **sessionId** column was used as a PARTITION KEY and the **itemInSession** column as CLUSTERING COLUMN.

In [69]:
# Query 1 - Create a table
query = "CREATE TABLE IF NOT EXISTS songs_by_session \
        (sessionId INT, itemInSession INT, artist VARCHAR, song VARCHAR, length FLOAT, \
        PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [70]:
# Code to set up the CSV file
file = 'event_datafile_new.csv'
'''
Opens the file using csv reader
Iterate over rows
Assign the INSERT statements 
Assign the columns
'''
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [71]:
# SELECT statement to verify the data was entered into the table
select_query = "SELECT artist, song, length \
        FROM songs_by_session \
        WHERE sessionId = 338 AND itemInSession = 4"

In [72]:
try:
    rows = session.execute(select_query)
except Exception as e:
    print(e) 

for row in rows:
    print("\n ARTIST: {} \n SONG: {} \n LENGTH: {}".format(row.artist, row.song, row.length))


 ARTIST: Faithless 
 SONG: Music Matters (Mark Knight Dub) 
 LENGTH: 495.30731201171875


### QUERY 2

Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

The **userId** and **sessionId** columns were used as a PARTITION KEY and the **itemInSession** column as CLUSTERING COLUMN.

In [73]:
# Query 2 - Create a table
query = "CREATE TABLE IF NOT EXISTS songs_by_user_session \
        (userId INT, sessionId INT, itemInSession INT, artist VARCHAR, song VARCHAR, \
        firstName VARCHAR, lastName VARCHAR, \
        PRIMARY KEY ((userId, sessionId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [74]:
# Code to set up the CSV file
file = 'event_datafile_new.csv'
'''
Opens the file using csv reader
Iterate over rows
Assign the INSERT statements 
Assign the columns
'''
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_by_user_session (userId, sessionId, itemInSession, artist, song, \
                    firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [75]:
# SELECT statement to verify the data was entered into the table
select_query = "SELECT artist, song, firstName, lastName \
        FROM songs_by_user_session \
        WHERE userId = 10 AND sessionId = 182"

In [76]:
try:
    rows = session.execute(select_query)
except Exception as e:
    print(e) 

for row in rows:
    print("\n ARTIST: {} \n SONG: {} \n FIRST NAME: {} \n LAST NAME: {}".format(row.artist, row.song, row.firstname, row.lastname))
    print('\n')


 ARTIST: Down To The Bone 
 SONG: Keep On Keepin' On 
 FIRST NAME: Sylvie 
 LAST NAME: Cruz



 ARTIST: Three Drives 
 SONG: Greece 2000 
 FIRST NAME: Sylvie 
 LAST NAME: Cruz



 ARTIST: Sebastien Tellier 
 SONG: Kilometer 
 FIRST NAME: Sylvie 
 LAST NAME: Cruz



 ARTIST: Lonnie Gordon 
 SONG: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 
 FIRST NAME: Sylvie 
 LAST NAME: Cruz




### QUERY 3

Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The **song** column was used as a PARTITION KEY and the **userId** column as CLUSTERING COLUMN.

In [77]:
# Query 3 - Create a table
query = "CREATE TABLE IF NOT EXISTS users_by_song \
        (song TEXT, userId INT, firstName TEXT, lastName Text, \
        PRIMARY KEY (song, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [78]:
# Code to set up the CSV file
file = 'event_datafile_new.csv'
'''
Opens the file using csv reader
Iterate over rows
Assign the INSERT statements 
Assign the columns
'''
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users_by_song (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [79]:
# SELECT statement to verify the data was entered into the table
select_query = "SELECT userId, firstName, lastName \
        FROM users_by_song \
        WHERE song='All Hands Against His Own'"

In [80]:
try:
    rows = session.execute(select_query)
except Exception as e:
    print(e) 

for row in rows:
    print("\n USERNAME: {}".format(row.firstname + ' ' + row.lastname))


 USERNAME: Jacqueline Lynch

 USERNAME: Tegan Levine

 USERNAME: Sara Johnson


### Drop the tables before closing out the sessions

In [81]:
# Drop the table before closing out the sessions
def drop_table(query):
    """
    Drops the table and handle errors
    """
    try:
        session.execute(query)
    except Exception as e:
        print(e)

In [82]:
drop_table("DROP TABLE songs_by_session")

drop_table("DROP TABLE songs_by_user_session")

drop_table("DROP TABLE users_by_song")

### Close the session and cluster connection¶

In [83]:
session.shutdown()
cluster.shutdown()